In [1]:
%env OMP_NUM_THREADS=10
%pylab inline
%load_ext autoreload
%autoreload 2

env: OMP_NUM_THREADS=10
Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
from imageio import imread
import cv2
import tensorflow as tf

LOCAL_DIR = '../'
if LOCAL_DIR not in sys.path:
    sys.path.append(LOCAL_DIR)

import photometric_augmentation as photoaug
import deform_augmentation as defaug
import improcess as imp
from jupyter_utils import display_image_batch

/home/ono/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Photometric Augmentations

In [3]:
sample_path = '../samples/happy_animal.jpg'
sample_image = imread(sample_path)

In [4]:
# tensorflow implementations
tf.reset_default_graph()
input_ph = tf.placeholder(tf.float32, [None,None,None])

primitives = photoaug.augmentations

photoaug_outputs = [input_ph]
for p in primitives:
    out = getattr(photoaug, p)(input_ph)
    photoaug_outputs.append(out)
photoaug_outputs = tf.concat([x[None] for x in photoaug_outputs], axis=0)

In [5]:
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True 
sess = tf.Session(config=tfconfig)
sess.run(tf.global_variables_initializer())

In [6]:
feed_dict = {
    input_ph: sample_image.astype(np.float32)
}
results = sess.run(photoaug_outputs, feed_dict=feed_dict)
display_image_batch(results)

../jupyter_utils.py:18: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if isinstance(batch, np.ndarray) and np.issubdtype(batch.dtype, np.float):


interactive(children=(IntSlider(value=3, description='idx', max=7), Output()), _dom_classes=('widget-interact'…

In [24]:
primitives

['additive_gaussian_noise',
 'additive_speckle_noise',
 'random_brightness',
 'random_contrast',
 'random_color_transform',
 'additive_shade',
 'motion_blur']

In [7]:
# # dump for README
# for name, img in zip(['original']+primitives, results.astype(np.uint8)):
#     imsave('../samples/doc/pa_{}.jpg'.format(name), img)

## Deforming Augmentations

In [9]:
sample_path = '../samples/happy_animal_grid.jpg'
sample_image = imread(sample_path).astype(np.float32)
num_examples = 8

In [17]:
homo_config = {
    'perspective': True,
    'scaling': True,
    'rotation': True,
    'translation': True,
    'patch_ratio': 0.8,
}

ela_size = 256

tf.reset_default_graph()
input_ph = tf.placeholder(tf.float32, [None,None,None])
# Homographic augmentation
homoaug_output = defaug.homographic_augmentation(input_ph, **homo_config)['image']
# Elastic deformation
elastic_output = defaug.elastic_deformation(input_ph, alpha=ela_size*2, sigma=ela_size*0.08, alpha_affine=ela_size*0.08)
# Random distortion
dist_output = defaug.random_distortion(input_ph[None], disable_border=True)[0]
# Euclid transform
euclid_output = defaug.euclid_augmentation(input_ph[None], max_rad=np.pi, max_scale=np.sqrt(2), max_shift=0.1, disable_border=True)[0]

In [18]:
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True 
sess = tf.Session(config=tfconfig)
sess.run(tf.global_variables_initializer())

In [20]:
stored_outputs = {
    'homographic': [sample_image],
    'elastic': [sample_image],
    'distortion': [sample_image],
    'euclid': [sample_image],
}
for i in range(num_examples):
    feed_dict = {
        input_ph: sample_image,
    }
    fetch_dict = {
        'homographic': homoaug_output,
        'elastic': elastic_output,
        'distortion': dist_output,
        'euclid': euclid_output,
    }
    outputs = sess.run(fetch_dict, feed_dict)
    
    for key, val in outputs.items():
        stored_outputs[key].append(val)

for key in stored_outputs.keys():
    stored_outputs[key] = np.array(stored_outputs[key])

In [21]:
display_image_batch(stored_outputs['homographic'])

../jupyter_utils.py:18: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if isinstance(batch, np.ndarray) and np.issubdtype(batch.dtype, np.float):


interactive(children=(IntSlider(value=4, description='idx', max=8), Output()), _dom_classes=('widget-interact'…

In [23]:
# # dump for README
# from jupyter_utils import save_as_gif
# for name, imgs in stored_outputs.items():
#     save_as_gif('../samples/doc/da_{}.gif'.format(name), imgs.astype(np.uint8), fps=2)